##### 问题:
我们有一些十分有用的方法，希望用它们来扩展其他类的功能。但是，需要添加方法
的这些类之间并不一定属于继承关系。因此，没法将这些方法直接关联到一个共同的
基类上。

##### 解决方法:
本节提到的问题在需要对类进行定制化处理时通常会出现。例如，某个库提供了一组
基础类以及一些可选的定制化方法，如果用户需要的话可以自行添加。

为了说明清楚，现在假设我们有兴趣将各式各样的定制化处理方法（例如，日志记录、
类型检查等）添加到映射型对象（mapping object）上。下面有一组 mixin 类来完成这
项任务：

In [24]:
class LoggedMappingMixin:
    # Add logging to get/set/delete operations for debugging.
    __slots__ = ()
    def __getitem__(self, key):
        print('Getting ' + str(key))
        return super().__getitem__(key)
    def __setitem__(self, key, value):
        print('Setting {} = {!r}'.format(key, value))
        return super().__setitem__(key, value)
    def __delitem__(self, key):
        print('Deleting ' + str(key))
        return super().__delitem__(key)

class SetOnceMappingMixin:
    # Only allow a key to be set once.
    __slots__ = ()
    def __setitem__(self, key, value):
        if key in self:
            raise KeyError(str(key) + ' already set')
        return super().__setitem__(key, value)
            
class StringKeysMappingMixin:
    # Restrict keys to strings only
    __slots__ = ()
    def __setitem__(self, key, value):
        if not isinstance(key, str):
            raise TypeError('keys must be strings')
        return super().__setitem__(key, value) 


这些类本身是无用的。实际上，如果实例化它们中的任何一个，一点儿有用的事情都
做不了（除了会产生异常之外）。相反，这些类存在的意义是要和其他映射型类通过多
重继承的方式混合在一起使用。示例如下：

In [25]:
class LoggedDict(LoggedMappingMixin, dict):
    pass

d = LoggedDict()
d['x'] = 23
print(d['x'] )

Setting x = 23
Getting x
23


In [26]:
from collections import defaultdict
class SetOnceDefaultDict(SetOnceMappingMixin, defaultdict):
    pass
d = SetOnceDefaultDict(list)
d['x'].append(2)
d['y'].append(3)
d['x'].append(10)
# d['x'] = 23 # KeyError: 'x already set'

In [27]:
from collections import OrderedDict
class StringOrderedDict(StringKeysMappingMixin,SetOnceMappingMixin,OrderedDict):
    pass
d = StringOrderedDict()
d['x'] = 23
# d[42] = 10 #TypeError: keys must be strings

在上面的示例中，可以发现这些 mixin 类和其他已有的类（例如：dict、defaultdict、
OrderedDict）结合在了一起。当它们混合在一起时，所有的类通过一起工作提供所需
的功能。

注意：<li>mixin 类绝不是为了直接实例化而创建的。例如，本节中所有的 mixin 类都不能
独自工作。它们必须同另一个实现了所需的映射功能的类混合在一起用才行。</li>
<li>mixin 类一般来说是没有状态的。这意味着 mixin 类没有__init__()方法，也没有
实例变量。在本节中，我们定义的__slots__ = ()就是一种强烈的提示，这表示 mixin 类
没有属于自己的实例数据。

如果考虑定义一个拥有__init__()方法以及实例变量的 mixin 类，请注意这里会有极大的
风险，因为这个类并不知道自己要和哪些其他的类混合在一起。因此，任何要创建出
的实例变量都必须以某种方式加以命名，以此避免出现命名冲突。此外，mixin 类的
\_\_init\_\_()方法必须要能合适地调用其他混合进来的类的__init__()方法。一般来说这很
难实现，因为不知道其他类的参数签名是什么。至少，我们必须得实现非常通用的参
数签名，这需要用到*arg、**kwargs。如果 mixin 类的__init__()方法自身还带有参数，
那么那些参数应该只能通过关键字来指定，并且在命名上还得和其他参数区分开，避
免命名冲突。对于定义了一个__init__()方法且接受一个关键字参数的 mixin 类，下面给
出了一种可能的实现方法：

In [28]:
class RestrictKeysMixin:
    def __init__(self, *args, _restrict_key_type, **kwargs):
        self.__restrict_key_type = _restrict_key_type
        super().__init__(*args, **kwargs)
    def __setitem__(self, key, value):
        if not isinstance(key, self.__restrict_key_type):
            raise TypeError('Keys must be ' + str(self.__restrict_key_type))
        super().__setitem__(key, value)

下面的例子展示了这个类应该如何使用：

In [29]:
class RDict(RestrictKeysMixin, dict):
    pass
d = RDict(_restrict_key_type=str)
e = RDict([('name','Dave'), ('n',37)], _restrict_key_type=str)
f = RDict(name='Dave', n=37, _restrict_key_type=str)
print(f) 

{'name': 'Dave', 'n': 37}

在这个例子中，可以注意到初始化 RDict()时仍然带有可被 dict()所接受的参数，但是有
一个额外的关键字参数 restrict_key_type 是提供给 mixin 类的。

最后，使用 super()函数是必要的，这也是编写 mixin 类的关键部分。在解决方案中，这
些类重新定义了一些特定的关键方法，比如__getitem__()和__setitem__()。但是，它们
也需要调用这些方法的原始版本。通过使用 super()，将这个任务转交给了方法解析顺
序（MRO）上的下一个类。本节中的这部分内容对于 Python 新手来说可能不是那么容
易理解，因为我们在没有父类的类中使用了 super()（初看上去感觉好像是个错误）。
但是，对于类似下面这样的类定义：

In [ ]:
class LoggedDict(LoggedMappingMixin, dict):
    pass

在LoggedMappingMixin 中使用super()函数会把任务转交到多重继承列表中的下一个类
上。也就是说，在 LoggedMappingMixin 中调用 super().\_\_getitem_\_()实际上会调用
dict.\_\_getitem\_\_()。如果没有这种行为，mixin 类根本没法正常工作。

实现 mixin 的另一种方法是利用类装饰器。例如，考虑如下的代码

In [ ]:
def LoggedMapping(cls):
    cls_getitem = cls.__getitem__
    cls_setitem = cls.__setitem__
    cls_delitem = cls.__delitem__
    def __getitem__(self, key):
        print('Getting ' + str(key))
        return cls_getitem(self, key)
    def __setitem__(self, key, value):
        print('Setting {} = {!r}'.format(key, value))
        return cls_setitem(self, key, value)
    def __delitem__(self, key):
        print('Deleting ' + str(key))
        return cls_delitem(self, key)
    cls.__getitem__ = __getitem__
    cls.__setitem__ = __setitem__
    cls.__delitem__ = __delitem__
    return cls

我们把这个函数作为装饰器添加到类定义上。例如：

In [ ]:
@LoggedMapping
class LoggedDict(dict):
    pass

如果试着这么做，就会发现能得到相同的行为，但是却完全不再涉及多重继承了。相
反，装饰器在这里只是对类定义做了一点点修改，从而替换掉特定的方法。有关类装
饰器的更多细节可在 9.12 节中找到。